# Imports

In [1]:
# to prevent to restart kernel when any changes are made to any imported file
%reload_ext autoreload
%autoreload 2

# to import any file from some other directory
# sys.path.append("/tmp/fastai/old")

# to stop printing warnings
import warnings
warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
warnings.warn = warn
    
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_colwidth', -1)

# to increase cells width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# to enable collapsible Headings and Functions
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
# !jupyter nbextensions_configurator enable --user
# !jupyter nbextension enable codefolding/main
# search collapsible to enable

# enable dark theme
# !pip install jupyterthemes
# !jt -t monokai

# monokai
# solarizedd
# !jt -r
import numpy as np
import faiss
from faiss import normalize_L2
import pandas as pd
import sys, os
from datetime import datetime, timedelta
import requests
import json
import timeit
from datetime import datetime
import os.path, time
from datetime import datetime, timedelta
from threading import Timer
import pdb, random

Loading faiss.


In [2]:
from get_glove_emb import get_word2vec_embeddings



Found 209 word vectors. 1


# Get Data

In [5]:
from random import randint


PATH="../real_not_real_kaggle/data/"
train=pd.read_csv(f"{PATH}train.csv").reset_index(drop=True)
test=pd.read_csv(f"{PATH}test.csv").reset_index(drop=True)
print(train.shape, test.shape)

# train["key"] = ["KEY"+x for x in list(map(str, random.sample(range(1000000, 9999999), len(train))))]
# test["key"]  = ["KEY"+x for x in list(map(str, random.sample(range(1000000, 9999999), len(test))))]
train["key"] = "KEY"+train.index.astype(str)
test["key"]  = "KEY"+test.index.astype(str)

# tr_embeddings = get_word2vec_embeddings(train, "tokens")
# ts_embeddings = get_word2vec_embeddings(test, "tokens")
# train["emb"]=list(tr_embeddings)
# test["emb"]=list(ts_embeddings)

# print(np.array(tr_embeddings).shape)
# print(np.array(ts_embeddings).shape)



(7613, 5) (3263, 4)


In [6]:
# test["key"].sample(200)
train.head()

,id,keyword,location,text,target,key
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,KEY0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,KEY1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,KEY2
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,KEY3
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,KEY4


In [7]:
test.shape, test.key.nunique()

((3263, 5), 3263)

In [8]:
# df=pd.read_pickle("/home/cloud_user/faisal/data/data_latest")
# df=df.rename(columns={"number": "key", "merged": "text"})
# print(df.shape)

In [9]:
def create_save_index(df, ids_col, xb_faiss_col, indx_fn, ids_fn):
    try:
        
        ids=np.array(df[ids_col].str.slice(start=3).tolist()).astype(int)
        xb_faiss = np.float32(np.asarray(df[xb_faiss_col].tolist()))

        d  = xb_faiss.shape[1] 
        print(d, type(d))

        normalize_L2(xb_faiss)

        index_faiss = faiss.IndexIDMap(faiss.IndexFlatIP(d))
        index_faiss.add_with_ids(xb_faiss, ids)

        print(index_faiss.is_trained)
        print(index_faiss.ntotal)

        faiss.write_index(index_faiss, indx_fn)
        np.save(ids_fn, ids)

        return True
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname =os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("Exception get_similar_tickets: ", e, exc_type, fname, exc_tb.tb_lineno)
        return False

In [10]:
import numpy as np
from scipy import spatial


def avg_feature_vector(sentence, num_features, word_index):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in word_index:
            n_words += 1
            feature_vec = np.add(feature_vec, word_index[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec





In [11]:
word_index=pd.read_pickle(f"{PATH}glove.840B.300d.pkl")

In [12]:
len(word_index.keys())

2196008

In [13]:
train["emb"] = train["text"].apply(lambda x: avg_feature_vector(x, 300, word_index)) 
test["emb"]  = test["text"].apply(lambda x: avg_feature_vector(x, 300, word_index)) 




In [14]:
train["emb"].shape

(7613,)

# Test Script

## Gensen

In [ ]:
create_save_index(df.sample(1000), "key", "emb", "index_gensen", "ids_gensen")

In [ ]:
result = get_similar_tickets(df.sample(2)[["key", "emb", "text"]], k=5, threshold=0.95)


## Glove

In [15]:
index_faiss=create_save_index(train, "key", "emb", "index_gloveee", "ids_gloveee")


300 <class 'int'>
True
7613


In [16]:
from similar_git import get_similar_tickets



True
7613


In [17]:
s=timeit.default_timer()
mydf = get_similar_tickets(test.sample(10)[["key", "emb", "text"]], k=5, threshold=0.95)
e=timeit.default_timer()
print("Overall total time: ", e-s)


function called
starting calculations
embedding col already present
query vector created !
Here we go ! <class 'faiss.swigfaiss.IndexIDMap'> (10, 300)
Got results from FAISS
create result dataframe
Overall total time:  0.165050211828202


In [44]:
for i in range(1,6):
    vals=mydf["NN"+str(i)+"_number"].tolist()
    if pd.isnull(vals).all():
        mydf["NN"+str(i)+"_text"]=np.nan
    else:
        mydf["NN"+str(i)+"_text"] = train.set_index("key").loc[mydf["NN"+str(i)+"_number"].tolist()].text.tolist()
final=mydf[["key", "query_text", "NN1_text","NN1_score", "NN1_number", "NN2_text","NN2_score", "NN2_number", "NN3_text","NN3_score", "NN3_number", "NN4_text","NN4_score", "NN4_number", "NN5_text","NN5_score","NN5_number", ]]
final


,key,query_text,NN1_text,NN1_score,NN1_number,NN2_text,NN2_score,NN2_number,NN3_text,NN3_score,NN3_number,NN4_text,NN4_score,NN4_number,NN5_text,NN5_score,NN5_number
0,KEY1967,#EPAO MARSAC reports on Manipur flood: Heavy incessant rains in Manipur have led to inundation causin... http://t.co/rh7m7T9Gd9 #MANIPUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KEY2602,A sinkhole grows ... in Brooklyn? http://t.co/UiqKDdVwKz http://t.co/aa8FwtOHyJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KEY3062,Medieval Upheaval (Hardy Boys: The Secret Files)\nFranklin W. Dixon - Aladdin. http://t.co/Yb7ijKtlnB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KEY575,hermancranston: WIRED : All these fires are burning through firefighters' budgets http://t.co/FSGAnfRgjH http://t.co/ju9zzu9TiOÛ_ Û_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KEY1509,#Fracking #Ecocide Eyewitness to Extreme Weather: 11 Social Media Posts that Show Just How Crazy T... http://t.co/dEdDH8Rme8 #Revolution,#ClimateChange Eyewitness to Extreme Weather: 11 Social Media Posts that Show Just How Crazy Things A... http://t.co/czpDn9oBiT #Anarchy,0.969349,KEY3516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,KEY183,Mourning notices for stabbing arson victims stir Û÷politics of griefÛª in Israel: Posters for Shira Banki and A... http://t.co/6o92wDfcLu,Mourning notices for stabbing arson victims stir Û÷politics of griefÛª in Israel: Posters for Shira Banki and A... http://t.co/3GZ5zQQTHe,1,KEY397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,KEY2459,To All The Meat-Loving Feminists Of The World Riot Grill Has Arrived http://t.co/MK9ZIp2ro7,To All The Meat-Loving Feminists Of The World Riot Grill Has Arrived http://t.co/TiOst8oKvX,1,KEY5758,To All The Meat-Loving Feminists Of The World Riot Grill Has Arrived http://t.co/SkAAUSjpO4 OliviaMiles01,1,KEY5766,To All The Meat-Loving Feminists Of The World Riot Grill HasåÊArrived http://t.co/uDQA53KfQu,0.973201,KEY5747,To All The Meat-Loving Feminists Of The World Riot Grill Has Arrived http://t.co/um3wTL5r7K #arts http://t.co/2LQyxZQ5DN,0.972885,KEY5745,NaN,NaN,NaN
7,KEY2406,Refugees as citizens - The Hindu http://t.co/GJSaAf3U6K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,KEY917,Microsoft Xbox 360 console RRoD red ring of death AS IS FOR PARTS OR REPAIR - Full read byÛ_ http://t.co/IpQCCT5hGC http://t.co/oofPEfRh3r,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,KEY1376,Leaving back to SF Friday have not packed one single thing 911 emergency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# issue in script:
#     test with gensen embeddings [Issues not in embeddings]

# issue in embeddings:
#     test with some other embeddings

# Test

In [ ]:


# def get_coefs(word,*arr): 
#     return word, np.asarray(arr, dtype='float32')

# def load_embedding(file):
#     if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
#         embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
#     else:
#         embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
#     return embeddings_index
# def make_embedding_matrix(embedding, tokenizer, len_voc):
#     all_embs = np.stack(embedding.values())
#     emb_mean,emb_std = all_embs.mean(), all_embs.std()
#     embed_size = all_embs.shape[1]
#     word_index = tokenizer.word_index
#     embedding_matrix = np.random.normal(emb_mean, emb_std, (len_voc, embed_size))
    
#     for word, i in word_index.items():
#         if i >= len_voc:
#             continue
#         embedding_vector = embedding.get(word)
#         if embedding_vector is not None: 
#             embedding_matrix[i] = embedding_vector
    
#     return embedding_matrix
# PATH="../real_not_real_kaggle/data/"


# glove = load_embedding(f'../real_not_real_kaggle/data/glove.6B.50d.txt')

In [ ]:



# glove_dir = f'{PATH}/' # This is the folder with the dataset

# glove_embedding = {} # We create a dictionary of word -> embedding
# f = open(f'{PATH}glove.6B.50d.txt') # Open file



In [ ]:
# len_voc = 10000
# # Tokenizing
# def make_tokenizer(texts, len_voc):
#     from keras.preprocessing.text import Tokenizer
#     t = Tokenizer(num_words=len_voc)
#     t.fit_on_texts(texts)
#     return t
# tokenizer = make_tokenizer(train['text'], len_voc)
# # Using TensorFlow backend.
# X = tokenizer.texts_to_sequences(train['text'])
# # I also apply padding, mostly to store X as an array.

# from keras.preprocessing.sequence import pad_sequences
# X = pad_sequences(X, 70)
# y = df['target'].values
# # For visualization, I'm gonna need to see which index corresponds to which word

# index_word = {0: ''}
# for word in tokenizer.word_index.keys():
#     index_word[tokenizer.word_index[word]] = word
# # Embedding Matrix
# embed_mat = make_embedding_matrix(glove, tokenizer, len_voc)

In [ ]:
# from nltk.corpus import stopwords
# from nltk.util import ngrams

# stop=set(stopwords.words('english'))

# from tqdm import tqdm
# from nltk.tokenize import word_tokenize
# from keras.preprocessing.text import Tokenizer

# df=train.copy()



# def create_corpus(df):
#     corpus=[]
#     for tweet in tqdm(df['text']):
#         words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
#         corpus.append(words)
#     return corpus
        
        
# corpus=create_corpus(df)
# # 100%|██████████| 10876/10876 [00:03<00:00, 3606.29it/s]
# embedding_dict={}
# with open(f'../real_not_real_kaggle/data/glove.6B.50d.txt','r') as f:
#     try:
#         for line in f:
#                 values=line.split()
#                 word=values[0]
#                 vectors=np.asarray(values[1:],'float32')
#                 embedding_dict[word]=vectors
#     except:
#         pass
# f.close()
# MAX_LEN=50
# print("step-1")
# tokenizer_obj=Tokenizer()
# print("step-2")


# tokenizer_obj.fit_on_texts(corpus)
# print("step-3")


# sequences=tokenizer_obj.texts_to_sequences(corpus)
# print("step-4")


# tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')
# word_index=tokenizer_obj.word_index
# print('Number of unique words:',len(word_index))
# # Number of unique words: 20342
# num_words=len(word_index)+1
# embedding_matrix=np.zeros((num_words,100))

# for word,i in tqdm(word_index.items()):
#     if i > num_words:
#         continue
    
#     emb_vec=embedding_dict.get(word)
#     if emb_vec is not None:
#         embedding_matrix[i]=emb_vec

        

In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential

max_features = 10000
maxlen = 25
embedding_size = 200
df=train.copy()


tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['headline']))
X = tokenizer.texts_to_sequences(df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = df['is_sarcastic']
# Using TensorFlow backend.
# EMBEDDING_FILE = '../input/glove-global-vectors-for-word-representation/glove.6B.200d.txt'
EMBEDDING_FILE = '../real_not_real_kaggle/data/glove.6B.50d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))

embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector